# Data Collection Proof-of-Concept
##  -- Mega-Cov Hydrator

The Google Colab notebook shows to to hydrate an Mega-cov archive.

You need to run this on Google Colab to get tweet metadata including its text.

The demo below hydrates 1 month of Mega-Cov data, filter it by Country == Canada & Language == English. 

It gets a total of 292 tweets in the end.

# Initializaition

In [79]:
from IPython.display import clear_output
%pip install twarc
%pip install jsonlines
# %pip install wget
%pip install jsonlines
clear_output()

import os
from google.colab import drive 
import pandas as pd
from datetime import datetime
# import wget
import json
import jsonlines
import tqdm
from twarc import Twarc


# shared archive -> "local" archive



Follow these steps:

1. Go to your Googe Drive
2. Create a folder named `MegaCov` 
3. Locate a Mega-cov archive file from [here](https://github.com/UBC-NLP/megacov/tree/master/tweet_ids) 
4. Add it to the `MegaCov` folder you just created in your Google Drive.


Now run the cell below.

When prompted, grant acess.

In [3]:

drive.mount('/content/gdrive')
working_directory = 'My Drive/MegaCov'

wd="/content/gdrive/"+working_directory
os.chdir(wd)

dirpath = os.getcwd()
print("current directory is : " + dirpath)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
current directory is : /content/gdrive/My Drive/MegaCov


# archive -> json

In [19]:
path = os.path.join(wd, "tweet_ids") 
if not os.path.exists('./tweet_ids'):
  try:
    os.makedirs(path)
    # os.chdir('tweet_ids')
  except:
    pass

In [65]:
#!pwd

In [ ]:
!tar -xvf  'MagaCOV_May_2020.tar.gz' -C './tweet_ids'
#todo unzip in batch *.tar.gz files

5_2020_1.json
5_2020_2.json


#  json  + filter -> tweet_ids

In [91]:
os.chdir('tweet_ids')

file = '5_2020_2.json'


In [92]:
twt_ids = []

#todo iterate over *.json in folder

with jsonlines.open(file) as reader:
    for obj in reader:
        #  filter by country and by Language:
        if obj['country'] == 'Canada' and obj['LangID_tool'] == 'en':
            twt_ids.append(obj['tweet_id'])

In [95]:
assert len(twt_ids) == 292
print('pass')

pass


In [96]:
assert twt_ids[0] == 1258551281041330176
print('pass')

pass


# tweet_ids -> txt

In [71]:
#@title Enter ID output file {run: "auto"}
os.chdir(wd)
final_tweet_ids_filename = "ids_CA_2020May_2.txt" #@param {type: "string"}
# The set of IDs is stored in this file.
with open(final_tweet_ids_filename, "w+") as f:
    for id in twt_ids:
        f.write('%s\n' % id)

# Hydrate 

## --using Twarc

In [68]:
#@title Insert API Keys here { run : "auto"}
from twarc import Twarc

# These keys are received after applying for a twitter developer account

consumer_key = "" #@param {type:"string"}
consumer_secret = "" #@param {type:"string"}
access_token = "" #@param {type:"string"}
access_token_secret = "" #@param {type:"string"}

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

In [85]:
#@title Set up Directory { run: "auto"}
input_filename_tweet_ids = "ids_CA_2020May_2.txt" #@param {type: "string"}
output_filename = "all-metadata-Canada-Engish-2020May_2.csv" #@param {type: "string"}

In [86]:
# Stores hydrated tweets here as jsonl objects
# Contains one json object per line
output_json_filename = output_filename[:output_filename.index(".")] + ".jsonl"
ids = []
with open(final_tweet_ids_filename, "r") as ids_file:
    ids = ids_file.read().split()
hydrated_tweets = []
ids_to_hydrate = set(ids)

# Looks at the output file for already hydrated tweets
if os.path.isfile(output_json_filename):
    with jsonlines.open(output_json_filename, "r") as reader:
        for i in reader.iter(type=dict, skip_invalid=True):
            # These tweets have already been hydrated. So remove them from ids_to_hydrate
            hydrated_tweets.append(i)
            ids_to_hydrate.remove(i["id_str"])
print("Total IDs: " + str(len(ids)) + ", IDs to hydrate: " + str(len(ids_to_hydrate)))
print("Hydrated: " + str(len(hydrated_tweets)))

count = len(hydrated_tweets)
start_index = count # The index from where tweets haven't been saved to the output_json_file
# Stores hydrated tweets to output_json_file every num_save iterations.
num_save  = 1000

# Now, use twarc and start hydrating
for tweet in t.hydrate(ids_to_hydrate):
    hydrated_tweets.append(tweet)
    count += 1
    # If num_save iterations have passed,
    if (count % num_save) == 0:
        # Open the output file
        # NOTE: Even if the code stops during IO, only tweets from the current iteration are lost.
        # Older tweets are preserved as the file is written in append mode.
        with jsonlines.open(output_json_filename, "a") as writer:
            print("Started IO")
            # Now write the tweets from start_index. The other tweets don't have to be written
            # as they were already written in a previous iteration or run.
            for hydrated_tweet in hydrated_tweets[start_index:]:
                writer.write(hydrated_tweet)
            print("Finished IO")
        print("Saved " + str(count) + " hydrated tweets.")
        # Now, since everything has been written. Reset start_index
        start_index = count
# There might be tweets unwritten in the last iteration if the count is not a multiple of num_tweets.
# In that case, just write out the remainder of tweets.
if count != start_index:
    print("Here with start_index", start_index)
    with jsonlines.open(output_json_filename, "a") as writer:
        for hydrated_tweet in hydrated_tweets[start_index:]:
           writer.write(hydrated_tweet)   

Total IDs: 292, IDs to hydrate: 292
Hydrated: 0
Here with start_index 0


In [87]:
# Convert jsonl to csv
import csv, jsonlines
output_json_filename = output_filename[:output_filename.index(".")] + ".jsonl"
# These are the column name that are selected to be stored in the csv
keyset = ["created_at", "id", "id_str", "full_text", "source", "truncated", "in_reply_to_status_id",
          "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str", 
          "in_reply_to_screen_name", "user", "coordinates", "place", "quoted_status_id",
          "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status", 
          "quote_count", "reply_count", "retweet_count", "favorite_count", "entities", 
          "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level", 
          "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright", 
          "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range",
          "quoted_status_permalink"]
hydrated_tweets = []
# Reads the current tweets
with jsonlines.open(output_json_filename, "r") as reader:
    for i in reader.iter(type=dict, skip_invalid=True):
        hydrated_tweets.append(i)
# Writes them out
with  open(output_filename, "w+") as output_file:
    d = csv.DictWriter(output_file, keyset)
    d.writeheader()
    d.writerows(hydrated_tweets)

Now the csv should be in MegaCov folder.